<a href="https://colab.research.google.com/github/SHASHANKBHATT03/PDF_Content_Analysis_and_Question_Generation/blob/main/PDF_Content_Analysis_and_Question_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Install dependencies
!pip install pdfplumber pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.9 MB/s eta 0:00:00


In [2]:
# STEP 2: Download the file from the provided direct link
import requests

# Download the file
file_url = 'https://drive.usercontent.google.com/u/0/uc?id=1YMeevGdKmkR_wDoc2AUadsnhaR75AghJ&export=download' # your link
local_pdf = 'input_assignment.pdf'

with requests.get(file_url, stream=True) as r:
    r.raise_for_status()
    with open(local_pdf, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("PDF downloaded as", local_pdf)

PDF downloaded as input_assignment.pdf


In [3]:
# STEP 3: Extract text and images

import os
import fitz  # pymupdf
import pdfplumber
import json

OUTPUT_DIR = '/content/output'
IMAGES_DIR = os.path.join(OUTPUT_DIR, 'images')
JSON_FILE = os.path.join(OUTPUT_DIR, 'questions.json')

os.makedirs(IMAGES_DIR, exist_ok=True)

# 3.1: Extract text page by page
question_texts = []
with pdfplumber.open(local_pdf) as pdf:
    for page in pdf.pages:
        text = page.extract_text() or ""
        question_texts.append(text.strip())

# 3.2: Extract images per page
pdf_doc = fitz.open(local_pdf)
option_images_per_page = []
for page_number in range(len(pdf_doc)):
    page = pdf_doc[page_number]
    image_list = page.get_images(full=True)
    images_on_this_page = []
    for idx, img in enumerate(image_list):
        xref = img[0]
        base_image = pdf_doc.extract_image(xref)
        image_bytes = base_image["image"]
        ext = base_image["ext"]
        img_filename = f"page{page_number+1}_option_image_{idx+1}.{ext}"
        img_path = os.path.join(IMAGES_DIR, img_filename)
        with open(img_path, "wb") as img_file:
            img_file.write(image_bytes)
        images_on_this_page.append(img_path)
    option_images_per_page.append(images_on_this_page)

# 3.3: Build output JSON
data = []
for i, question_text in enumerate(question_texts):
    question_obj = {
        "question": question_text,
        "option_images": option_images_per_page[i]
    }
    data.append(question_obj)

os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(JSON_FILE, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

print(f"✅ Extraction complete!")
print(f"Images: {IMAGES_DIR}")
print(f"Questions JSON: {JSON_FILE}")


✅ Extraction complete!
Images: /content/output/images
Questions JSON: /content/output/questions.json


In [4]:
import shutil
from google.colab import files

# Zip the output folder
shutil.make_archive("output", 'zip', OUTPUT_DIR)

# Download the zip file
files.download("output.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>